In [35]:
import pandas as pd
import geopandas as gpd
import rasterio
import rasterio.features
import rasterio.warp
from rasterio.plot import show
#Importing the Nominatim geocoder class 
#from geopy.geocoders import Nominatim
from shapely.geometry import point, Polygon
import rioxarray
from pyproj import CRS
from mpl_toolkits import mplot3d
import numpy as np
import matplotlib.pyplot as plt
import requests
from osgeo import gdal
from osgeo import ogr
import fiona
%matplotlib inline
from shapely.geometry import Polygon
import shapefile

In [36]:
address = input('Provide a valid address in Flanders: ')
#Heikantstraat 124, 2910 Essen
#Epicealaan 28, 2910 Essen
#Moerkantsebaan 34, 2910 Essen
#Maststraat 2, 2910 Essen

Provide a valid address in Flanders: 


In [37]:
# get a polygoon based on address in Flanders

def get_coordinates(address: str):
    req = requests.get(f"https://loc.geopunt.be/v4/Location?q={address}").json()
    info = {'address' : address, 
                'x_value' : req['LocationResult'][0]['Location']['X_Lambert72'],
                'y_value' : req['LocationResult'][0]['Location']['Y_Lambert72'],
                'street' : req['LocationResult'][0]['Thoroughfarename'],
                'house_number' : req['LocationResult'][0]['Housenumber'], 
                'postcode': req['LocationResult'][0]['Zipcode'], 
                'municipality' : req['LocationResult'][0]['Municipality']}
    
    detail = requests.get("https://api.basisregisters.vlaanderen.be/v1/adresmatch", 
                          params={"postcode": info['postcode'], 
                                  "straatnaam": info['street'],
                                  "huisnummer": info['house_number']}).json()
    building = requests.get(detail['adresMatches'][0]['adresseerbareObjecten'][0]['detail']).json()
    build = requests.get(building['gebouw']['detail']).json()
    info['polygon'] = [build['geometriePolygoon']['polygon']]
    return info['polygon'][0]['coordinates'][0] 

In [38]:
# store polygon in a variable
polygon = get_coordinates(address)
polygon

IndexError: list index out of range

In [39]:
x_polygon = [i[0] for i in polygon]
x_polygon

[155223.95828686655,
 155213.6574228555,
 155171.2693908289,
 155173.66260682791,
 155164.09172682464,
 155166.7246228233,
 155165.0949908197,
 155165.15828682482,
 155150.4937108159,
 155151.53620681167,
 155143.31067080796,
 155146.09659080952,
 155168.936590828,
 155169.29281482846,
 155180.87329483032,
 155180.56359883398,
 155223.95828686655]

In [40]:
y_polygon = [i[1] for i in polygon]
y_polygon

[238871.00343171507,
 238914.83779974654,
 238906.44547973946,
 238893.92829573154,
 238892.03107972816,
 238879.8068237193,
 238879.46730371937,
 238879.06019971892,
 238876.21552771702,
 238871.08951171488,
 238869.4486797154,
 238855.5521037057,
 238860.08458370715,
 238858.41258370504,
 238860.7199757062,
 238862.39191170782,
 238871.00343171507]

In [41]:
polygon_shapely = Polygon(polygon)
print(polygon_shapely)

POLYGON ((155223.9582868665 238871.0034317151, 155213.6574228555 238914.8377997465, 155171.2693908289 238906.4454797395, 155173.6626068279 238893.9282957315, 155164.0917268246 238892.0310797282, 155166.7246228233 238879.8068237193, 155165.0949908197 238879.4673037194, 155165.1582868248 238879.0601997189, 155150.4937108159 238876.215527717, 155151.5362068117 238871.0895117149, 155143.310670808 238869.4486797154, 155146.0965908095 238855.5521037057, 155168.936590828 238860.0845837072, 155169.2928148285 238858.412583705, 155180.8732948303 238860.7199757062, 155180.563598834 238862.3919117078, 155223.9582868665 238871.0034317151))


In [42]:
left = min(x_polygon)
right = max(x_polygon)
top = max(y_polygon)
bottom = min(y_polygon)
print(f"left = {left}, right = {right}, top = {top}, bottom = {bottom}")

left = 155143.31067080796, right = 155223.95828686655, top = 238914.83779974654, bottom = 238855.5521037057


In [43]:
type(left)

float

In [44]:
# create a rectangle which contains the polygoon (left,bottom, right, top)

In [45]:
polygon_bounds = [[left,bottom, right, top]]
polygon_bounds

[[155143.31067080796,
  238855.5521037057,
  155223.95828686655,
  238914.83779974654]]

In [46]:
# find to which file belong polygon_bounds - loop over the the DTM_bounds.csv

In [47]:
df = pd.read_csv('DTM_bounds.csv')

In [48]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43 entries, 0 to 42
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   url     43 non-null     object 
 1   left    43 non-null     float64
 2   right   43 non-null     float64
 3   top     43 non-null     float64
 4   bottom  43 non-null     float64
dtypes: float64(4), object(1)
memory usage: 1.8+ KB


In [49]:
df.head()

,url,left,right,top,bottom
0,zip+https://downloadagiv.blob.core.windows.net...,145000.0,162000.0,247000.0,238000.0
1,zip+https://downloadagiv.blob.core.windows.net...,162000.0,194000.0,250000.0,238000.0
2,zip+https://downloadagiv.blob.core.windows.net...,194000.0,206000.0,248000.0,238000.0
3,zip+https://downloadagiv.blob.core.windows.net...,53000.0,66000.0,225500.0,218000.0
4,zip+https://downloadagiv.blob.core.windows.net...,66000.0,98000.0,232000.0,218000.0


In [51]:
for index, row in df.iterrows():
    if left >= row.left and right <= row.right and top <= row.top and bottom >= row.bottom:
        print(row.url)

zip+https://downloadagiv.blob.core.windows.net/dhm-vlaanderen-ii-dtm-raster-1m/DHMVIIDTMRAS1m_k01.zip!/GeoTIFF/DHMVIIDTMRAS1m_k01.tif
